 Effects of Policy on the Housing Market ** 

In [22]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


import matplotlib.pyplot as plt
import seaborn as sns


In [23]:
df = pd.read_csv("effects-of-policy-on-the-housing-market/train.csv")
df.head()


,Unnamed: 0.1,Unnamed: 0,id,name,neighborhood_overview,host_id,host_name,host_response_time,host_response_rate,host_acceptance_rate,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,monthly_revenue
0,0,879,19792418,Home in Vancouver · ★4.75 · 1 bedroom · 1 bed ...,Everything you need is nearby. <br /><br />Hig...,57488206,Jessi,NaN,NaN,NaN,...,4.80,4.82,4.90,4.87,4.69,4.81,f,3,0.77,2108
1,1,6416,1015650685503221866,Guest suite in Vancouver · ★New · 2 bedrooms ·...,NaN,139792573,Daniel,within a few hours,100%,100%,...,NaN,NaN,NaN,NaN,NaN,NaN,f,1,NaN,2730
2,2,1820,35265562,Guest suite in Vancouver · ★4.85 · 2 bedrooms ...,Beautiful neighbourhood close to prosperous Ma...,265504225,Alex,within an hour,100%,98%,...,4.90,4.78,4.97,4.94,4.90,4.75,f,1,3.22,2254
3,3,5346,911948980885194155,Home in Vancouver · ★5.0 · 1 bedroom · 1 bed ·...,We are located in a quiet residential neighbor...,22595056,Raymond,NaN,NaN,92%,...,5.00,5.00,5.00,5.00,4.86,5.00,f,1,1.28,3187
4,4,2484,46069251,Guest suite in Vancouver · ★4.93 · 1 bedroom ·...,Kitsilano at it's best! Short walk to all the ...,65683877,Yendi,within an hour,100%,95%,...,4.93,4.89,4.97,4.97,4.96,4.85,f,1,2.01,3479


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5352 entries, 0 to 5351
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0.1                    5352 non-null   int64  
 1   Unnamed: 0                      5352 non-null   int64  
 2   id                              5352 non-null   int64  
 3   name                            5352 non-null   object 
 4   neighborhood_overview           3476 non-null   object 
 5   host_id                         5352 non-null   int64  
 6   host_name                       5352 non-null   object 
 7   host_response_time              4263 non-null   object 
 8   host_response_rate              4263 non-null   object 
 9   host_acceptance_rate            4620 non-null   object 
 10  host_is_superhost               5315 non-null   object 
 11  host_listings_count             5352 non-null   int64  
 12  host_total_listings_count       53

Getting Feature Importance

In [25]:
# Extract the first 20 columns and the 43rd column (index 42) from the DataFrame
df_first_selected = df.iloc[:, list(range(20))].copy()

# Display the result
df_first_selected.head()


,Unnamed: 0.1,Unnamed: 0,id,name,neighborhood_overview,host_id,host_name,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates
0,0,879,19792418,Home in Vancouver · ★4.75 · 1 bedroom · 1 bed ...,Everything you need is nearby. <br /><br />Hig...,57488206,Jessi,NaN,NaN,NaN,f,3,3,"Vancouver, British Columbia, Canada",Hastings-Sunrise,49.283570,-123.056490,Entire home,Entire home/apt,2
1,1,6416,1015650685503221866,Guest suite in Vancouver · ★New · 2 bedrooms ·...,NaN,139792573,Daniel,within a few hours,100%,100%,f,1,4,NaN,Sunset,49.221120,-123.092320,Entire guest suite,Entire home/apt,5
2,2,1820,35265562,Guest suite in Vancouver · ★4.85 · 2 bedrooms ...,Beautiful neighbourhood close to prosperous Ma...,265504225,Alex,within an hour,100%,98%,t,1,1,"Vancouver, British Columbia, Canada",Riley Park,49.252140,-123.098170,Entire guest suite,Entire home/apt,6
3,3,5346,911948980885194155,Home in Vancouver · ★5.0 · 1 bedroom · 1 bed ·...,We are located in a quiet residential neighbor...,22595056,Raymond,NaN,NaN,92%,t,1,1,"Vancouver, British Columbia, Canada",Sunset,49.221709,-123.097762,Entire home,Entire home/apt,2
4,4,2484,46069251,Guest suite in Vancouver · ★4.93 · 1 bedroom ·...,Kitsilano at it's best! Short walk to all the ...,65683877,Yendi,within an hour,100%,95%,t,2,3,"Vancouver, British Columbia, Canada",Kitsilano,49.258980,-123.161790,Entire guest suite,Entire home/apt,4


In [26]:

# Extract the last 20 columns and the 43rd column (index 42) from the DataFrame
df_second_selected = df.iloc[:, 22:44].copy()

# Display the result
df_second_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5352 entries, 0 to 5351
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   price                           4687 non-null   object 
 1   minimum_nights                  5352 non-null   int64  
 2   maximum_nights                  5352 non-null   int64  
 3   minimum_nights_avg_ntm          5352 non-null   float64
 4   maximum_nights_avg_ntm          5352 non-null   float64
 5   availability_30                 5352 non-null   int64  
 6   availability_60                 5352 non-null   int64  
 7   availability_90                 5352 non-null   int64  
 8   availability_365                5352 non-null   int64  
 9   number_of_reviews               5352 non-null   int64  
 10  number_of_reviews_ltm           5352 non-null   int64  
 11  review_scores_rating            4452 non-null   float64
 12  review_scores_accuracy          44

In [27]:
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'id', 'host_id', 'host_name', 'name'], inplace=True)

In [28]:
numerical_cols = ['host_response_rate', 'host_acceptance_rate', 'host_listings_count', 'host_total_listings_count', 'latitude', 'longitude', 'accommodates', 'monthly_revenue']
categorical_cols = ['host_response_time', 'neighbourhood', 'host_is_superhost', 'neighbourhood_cleansed', 'property_type', 'room_type']

In [29]:
X_train = df.drop(columns=["monthly_revenue"])
y_train = df["monthly_revenue"]

In [30]:
# Mapping response time
response_time_mapping = {
    "within an hour": 4,
    "within a few hours": 3,
    "within a day": 2,
    "a few days or more": 1,
    None: 0  # For missing values
}

df['host_response_time'] = df['host_response_time'].map(response_time_mapping)
df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').astype(float) / 100
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip('%').astype(float) / 100


df.head()

,neighborhood_overview,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,neighbourhood,neighbourhood_cleansed,latitude,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,monthly_revenue
0,Everything you need is nearby. <br /><br />Hig...,NaN,NaN,NaN,f,3,3,"Vancouver, British Columbia, Canada",Hastings-Sunrise,49.283570,...,4.80,4.82,4.90,4.87,4.69,4.81,f,3,0.77,2108
1,NaN,3.0,1.0,1.00,f,1,4,NaN,Sunset,49.221120,...,NaN,NaN,NaN,NaN,NaN,NaN,f,1,NaN,2730
2,Beautiful neighbourhood close to prosperous Ma...,4.0,1.0,0.98,t,1,1,"Vancouver, British Columbia, Canada",Riley Park,49.252140,...,4.90,4.78,4.97,4.94,4.90,4.75,f,1,3.22,2254
3,We are located in a quiet residential neighbor...,NaN,NaN,0.92,t,1,1,"Vancouver, British Columbia, Canada",Sunset,49.221709,...,5.00,5.00,5.00,5.00,4.86,5.00,f,1,1.28,3187
4,Kitsilano at it's best! Short walk to all the ...,4.0,1.0,0.95,t,2,3,"Vancouver, British Columbia, Canada",Kitsilano,49.258980,...,4.93,4.89,4.97,4.97,4.96,4.85,f,1,2.01,3479


In [31]:
numerical_cols = ['minimum_nights', 'maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_30', 
                  'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 'number_of_reviews_ltm', 
                  'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
                  'review_scores_communication', 'review_scores_location', 'review_scores_value', 'calculated_host_listings_count', 
                  'reviews_per_month']

categorical_cols = ['host_response_time', 'host_is_superhost', 'neighbourhood', 'neighbourhood_cleansed', 'property_type', 'room_type', 'instant_bookable']



In [32]:
preprocessor = make_column_transformer(
    (Pipeline([
        ('imputer', SimpleImputer(strategy='mean')), 
        ('scaler', StandardScaler())  
    ]), numerical_cols),
    
    (Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')), 
        ('onehot', OneHotEncoder(handle_unknown='ignore')) 
    ]), categorical_cols)
)

In [33]:
X_train = df[numerical_cols + categorical_cols].copy()


# Verify the result
print("Columns in X_train:", X_train.columns)



y_train = df['monthly_revenue']  

Columns in X_train: Index(['minimum_nights', 'maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',
       'calculated_host_listings_count', 'reviews_per_month',
       'host_response_time', 'host_is_superhost', 'neighbourhood',
       'neighbourhood_cleansed', 'property_type', 'room_type',
       'instant_bookable'],
      dtype='object')


In [34]:
from sklearn.ensemble import RandomForestRegressor
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  
    ('model', RandomForestRegressor(random_state=42)) 
])

In [35]:
model_pipeline.fit(X_train, y_train)

print(len(X_train))  # Should return something like (num_samples, num_features)
print(y_train.shape)  # Should return something like (num_samples,)


5352
(5352,)


In [36]:
rf_model = model_pipeline.named_steps['model']

# Get the feature importances
feature_importance = rf_model.feature_importances_
print(len(X_train.columns))  # Should match the length of feature_importance
print(len(feature_importance))

# Create a DataFrame to display feature importance
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

26
107


ValueError: All arrays must be of the same length

In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

test_df = pd.read_csv("effects-of-policy-on-the-housing-market/test.csv")

predictions = model_pipeline.predict(test_df)

# Create a new DataFrame with only 'id' and 'monthly_Revenue' prediction
result_df = pd.DataFrame({
    'id': test_df['id'],  # Assuming 'id' is a column in your test_df
    'monthly_Revenue': predictions
})

result_df.to_csv('monthly_revenue_predictions.csv', index=False)



# mse = mean_squared_error(y_val, y_pred)
# mae = mean_absolute_error(y_val, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'Mean Squared Error (MSE): {mse}')
# print(f'Mean Absolute Error (MAE): {mae}')
# print(f'R² Score: {r2}')